In [1]:
import requests
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import random
import pandas as pd
from nerd import ner
import threading
from concurrent import futures

In [2]:
f = open("urls.txt", 'r')
urls = f.read().rstrip().split('\n')
# print(urls)
f.close()
print(len(urls))
content = {
    'LINK': [],
    'PERSON GETTING INTERVIEWED': [],
    'PODCASTS': [],
    'HOST': []
}

df = pd.read_csv('data.tsv', sep = '\t').to_dict()
for key in df.keys():
    content[str(key)] = list(df[key].values())

6963


In [3]:
opts = Options()
opts.add_argument("--window-size=1920,1080")
# opts.add_argument("--headless")
opts.add_argument("--detach")
opts.add_argument("user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36")
driver = Chrome(executable_path="./chromedriver", options=opts)


In [4]:
def crawl(url):
    try:
        global content
        driver.get(url)
    #     time.sleep(3)

        total = driver.find_element_by_xpath('//div[@class="product-artwork__caption small-hide medium-show"]')
        total = total.text.split(' ')[0].split('\n')[0]
        total = total.replace('.', '')
        total = total.replace(',', '')
        total = int(total)

        podcast = driver.find_element_by_xpath('//span[@class="product-header__title"]').text
        host = driver.find_element_by_xpath('//span[@class="product-header__identity podcast-header__identity"]').text

        cnt = len(driver.find_elements_by_xpath('//a[@class="link tracks__track__link--block"]'))
        while len(driver.find_elements_by_xpath('//button[@class="link"]')) != 0:
            button = driver.find_element_by_xpath('//button[@class="link"]')
            cnt += int(button.text.split(' ')[1])
            button.click()
            time.sleep(2)
        print(f"Scroll finish: {cnt} - {podcast} - {total}")

        blocks = driver.find_elements_by_xpath('//div[@class="l-column  small-valign-top tracks__track__content"]')
        print(podcast, len(blocks))
        cnt = 0
        for block in blocks:
            if len(block.find_elements_by_xpath('.//div[@class="we-truncate we-truncate--multi-line  tracks__track__copy"]')) == 0: 
                continue
            chunk = block.find_element_by_xpath('.//div[@class="we-truncate we-truncate--multi-line  tracks__track__copy"]').text
            chunk = chunk[:150]
            chunk = chunk.replace('/w', 'with ')
            chunk = chunk.replace('/', ' ')
            chunk = chunk.replace(' - ', ' . ')
            chunk = chunk.replace(' w ', ' with ')

            doc = ner.name(chunk, language='en_core_web_sm')
            text_label = [(X.text, X.label_) for X in doc]

            ok = False
            li = []
            if len(text_label) != 0:
                for each in reversed(text_label):
                    if each[1] == 'PERSON' and each[0].upper() not in host.upper() and len(each[0].split(' ')) > 1:
                        ok = True
                        li.append(each[0])
                        
            if ok: 
                for each in li:
                    content['LINK'].append(block.find_element_by_xpath('./a').get_attribute('href'))
                    content['PERSON GETTING INTERVIEWED'].append(each)
                    content['PODCASTS'].append(podcast)
                    content['HOST'].append(host)
                    cnt+=1
                    
        print(f"Finish collected: {cnt}/{len(blocks)} - {podcast}") 
        print(f"Curren total: {len(content['LINK'])}")
    except:
        print("!!!!ERROR!!!!")
        time.sleep(60)
        crawl(url)

In [5]:
for url in urls:
    crawl(url)
driver.close()

Scroll finish: 35 - Revolutionary Rising - 35
Revolutionary Rising 35
Finish collected: 1/35 - Revolutionary Rising
Curren total: 37910
Scroll finish: 87 - How I Grew This - 87
How I Grew This 87
Finish collected: 43/87 - How I Grew This
Curren total: 37953
Scroll finish: 44 - Winning Digital Customers - 44
Winning Digital Customers 44
Finish collected: 38/44 - Winning Digital Customers
Curren total: 37991
Scroll finish: 10 - Adventist Peace Radio - 10
Adventist Peace Radio 10
Finish collected: 9/10 - Adventist Peace Radio
Curren total: 38000
Scroll finish: 106 - Subscription Box Basics with Julie Ball - 106
Subscription Box Basics with Julie Ball 106
Finish collected: 45/106 - Subscription Box Basics with Julie Ball
Curren total: 38045
Scroll finish: 296 - Autoline This Week - Video - 598
Autoline This Week - Video 286
Finish collected: 58/286 - Autoline This Week - Video
Curren total: 38103
Scroll finish: 35 - Meet the RIA - 35
Meet the RIA 35
Finish collected: 28/35 - Meet the RIA
C

Finish collected: 14/13 - Staring at the World with BoDeans’ Kurt Neumann
Curren total: 40420
Scroll finish: 86 - Ignite Your Business Radio Show - 86
Ignite Your Business Radio Show 86
Finish collected: 36/86 - Ignite Your Business Radio Show
Curren total: 40456
Scroll finish: 173 - Inventors Helping Inventors - 173
Inventors Helping Inventors 173
Finish collected: 282/173 - Inventors Helping Inventors
Curren total: 40738
Scroll finish: 100 - Amplify Your Success - 100
Amplify Your Success 100
Finish collected: 53/100 - Amplify Your Success
Curren total: 40791
Scroll finish: 100 - The Steffan Tubbs Show Podcast - 100
The Steffan Tubbs Show Podcast 100
Finish collected: 83/100 - The Steffan Tubbs Show Podcast
Curren total: 40874
Scroll finish: 61 - Disruptive Forces in Investing - 61
Disruptive Forces in Investing 61
Finish collected: 4/61 - Disruptive Forces in Investing
Curren total: 40878
Scroll finish: 280 - Resourceful Designer: Strategies for running a graphic design business - 2

Finish collected: 16/140 - My Digital Farmer | Marketing Strategies for Farmers
Curren total: 43133
Scroll finish: 86 - Be Impactful by Impact Fashion - 100
Be Impactful by Impact Fashion 76
Finish collected: 45/76 - Be Impactful by Impact Fashion
Curren total: 43178
Scroll finish: 166 - 5 Questions With Dan Schawbel - 166
5 Questions With Dan Schawbel 166
Finish collected: 168/166 - 5 Questions With Dan Schawbel
Curren total: 43346
Scroll finish: 77 - Be In Demand with Laurie-Ann Murabito - 77
Be In Demand with Laurie-Ann Murabito 77
Finish collected: 6/77 - Be In Demand with Laurie-Ann Murabito
Curren total: 43352
Scroll finish: 100 - Michael Campbell's Money Talks - 100
Michael Campbell's Money Talks 100
Finish collected: 44/100 - Michael Campbell's Money Talks
Curren total: 43396
Scroll finish: 35 - Success Fundamentals - 35
Success Fundamentals 35
Finish collected: 38/35 - Success Fundamentals
Curren total: 43434
Scroll finish: 250 - a16z Podcast - 250
a16z Podcast 250
Finish coll

Scroll finish: 100 - Lab Coat Agents Podcast - 100
Lab Coat Agents Podcast 100
Finish collected: 87/100 - Lab Coat Agents Podcast
Curren total: 46965
Scroll finish: 56 - Legends of Sales and Marketing - 56
Legends of Sales and Marketing 56
Finish collected: 34/56 - Legends of Sales and Marketing
Curren total: 46999
Scroll finish: 24 - BYO Nano Brew Podcast - 24
BYO Nano Brew Podcast 24
Finish collected: 10/24 - BYO Nano Brew Podcast
Curren total: 47009
Scroll finish: 290 - Dietitian Boss with Libby Rothschild MS, RD, CPT - 290
Dietitian Boss with Libby Rothschild MS, RD, CPT 290
Finish collected: 165/290 - Dietitian Boss with Libby Rothschild MS, RD, CPT
Curren total: 47174
Scroll finish: 37 - Healthcare is Hard: A Podcast for Insiders - 37
Healthcare is Hard: A Podcast for Insiders 37
Finish collected: 18/37 - Healthcare is Hard: A Podcast for Insiders
Curren total: 47192
Scroll finish: 47 - Nurse Leader Network - 47
Nurse Leader Network 47
Finish collected: 12/47 - Nurse Leader Netwo

Scroll finish: 75 - Average Joe Finances - 75
Average Joe Finances 75
Finish collected: 109/75 - Average Joe Finances
Curren total: 48746
Scroll finish: 73 - Grant Writing Simplified - 73
Grant Writing Simplified 73
Finish collected: 19/73 - Grant Writing Simplified
Curren total: 48765
Scroll finish: 259 - SweetLife Entrepreneur™ Podcast - 259
SweetLife Entrepreneur™ Podcast 259
Finish collected: 66/259 - SweetLife Entrepreneur™ Podcast
Curren total: 48831
Scroll finish: 127 - Farm4Profit Podcast - 127
Farm4Profit Podcast 127
Finish collected: 80/127 - Farm4Profit Podcast
Curren total: 48911
Scroll finish: 106 - DEFI TIMES - 106
DEFI TIMES 99
!!!!ERROR!!!!


KeyboardInterrupt: 

In [8]:
# print(content)
df = pd.DataFrame(content)
df.sort_values(by = ["PERSON GETTING INTERVIEWED"])
df.drop_duplicates("PERSON GETTING INTERVIEWED", keep = False, inplace = True)
df.sort_values("PODCASTS")
df.to_csv('data.tsv', index = False, sep = "\t")
print(df)

                                                    LINK  \
0      https://podcasts.apple.com/us/podcast/superspr...   
1      https://podcasts.apple.com/us/podcast/the-elit...   
2      https://podcasts.apple.com/us/podcast/the-elit...   
3      https://podcasts.apple.com/us/podcast/the-1-bi...   
4      https://podcasts.apple.com/us/podcast/the-1-bi...   
...                                                  ...   
42147  https://podcasts.apple.com/us/podcast/5-solana...   
42148  https://podcasts.apple.com/us/podcast/harmony-...   
42149  https://podcasts.apple.com/us/podcast/betting-...   
42150  https://podcasts.apple.com/us/podcast/mgh-bett...   
42151  https://podcasts.apple.com/us/podcast/the-pote...   

      PERSON GETTING INTERVIEWED  \
0                  Bitpiggys Jim   
1              Cedric Youngelman   
2                    Craig Shipp   
3                Trevor Lawrence   
4               Robert Valentine   
...                          ...   
42147                Austin

In [7]:
f = open("compare.txt", "r")
text = f.read()
test = text.rstrip().split('\n')
f.close()

deleted = 0
for i in range(len(content['PERSON GETTING INTERVIEWED'])):
    word = content['PERSON GETTING INTERVIEWED'][i - deleted]
    _deleted = 0
    for j in range(i-deleted):
        if j>=0:
            if word in content['PERSON GETTING INTERVIEWED'][j-_deleted] or content['PERSON GETTING INTERVIEWED'][j-_deleted] in word:
#                 print(word, " - ", content['PERSON GETTING INTERVIEWED'][j-_deleted], "-", i, j+deleted)
                for key in content.keys():
                    del content[key][j-_deleted]
                deleted+=1
                _deleted+=1

deleted = 0
for i in range(len(content['PERSON GETTING INTERVIEWED'])):
    word = content['PERSON GETTING INTERVIEWED'][i - deleted] 

    word = word.replace('Interview','')
    word = word.replace('Interviews','')
    word = word.replace('Access','')
    word = word.replace('Visit','')
    word = word.replace('Meet ','')
    word = word.replace('Click','')
    word = word.replace('Learned','')
    word = word.replace('Learn','')
    word = word.replace('Join','')
    word = word.replace(' Links','')
    word = word.replace('Links ','')
    word = word.replace('Subscribe','')
    word = word.replace('Messenger','')
    word = word.replace('Message','')
    word = word.replace('Follow','')
    word = word.replace('Contact','')
    word = word.replace('Download', '')
    word = word.replace('Support', '')
    word = word.replace('Google', '')
    word = word.replace('Facebook', '')
    word = word.replace('Twitter', '')
    word = word.replace('Instagram', '')
    word = word.replace('Tesla', '')
    word = word.replace('Ethereum', '')
    word = word.replace('Bitcoin', '')
    word = word.replace('Tik Tok', '')
    word = word.replace('Online', '')
    word = word.replace('Offline', '')
    word = word.replace('Ask', '')
    word = word.replace('Answer', '')
    word = word.replace('Question', '')
    word = word.replace('Founder', '')
    word = word.replace('Owner', '')
    word = word.replace('Director', '')
    word = word.replace(' Says', '')
    word = word.replace('Says ', '')
    word = word.replace('Sharing', '')
    word = word.replace('Looking', '')
    word = word.replace('Looks ', '')
    word = word.replace('Shares', '')
    word = word.replace('Connect', '')
    word = word.replace('Crypto', '')
    word = word.replace('Currency', '')
    word = word.replace('My ', '')
    word = word.replace('His ', '')
    word = word.replace('Her ', '')
    word = word.replace('Their ', '')
    word = word.replace('Our ', '')
    word = word.replace('Uncle ', '')
    word = word.replace('Dad ', '')
    word = word.replace('Aunt ', '')
    word = word.replace(' Us', '')
    word = word.replace('Grandmother', '')
    word = word.replace('Grandfather', '')
    word = word.replace('Officer', '')
    word = word.replace('Billionare', '')
    word = word.replace('Millionare', '')
    word = word.replace('Consultan', '')
    word = word.replace('Economi', '')
    word = word.replace('Producing', '')
    word = word.replace('Platform', '')
    word = word.replace('Framework', '')
    word = word.replace('Science', '')
    word = word.replace('Workshop', '')
    word = word.replace('Build ', '')
    word = word.replace('Building', '')
    word = word.replace('Yourself', '')
    word = word.replace('Myself', '')
    word = word.replace('Himself', '')
    word = word.replace('Themselves', '')
    word = word.replace('Ourselves', '')
    word = word.replace('Herself', '')
    word = word.replace('Which ', '')
    word = word.replace('What ', '')
    word = word.replace('When ', '')
    word = word.replace('How ', '')
    word = word.replace('Where ', '')
    word = word.replace('Conversation', '')
    word = word.replace('Commerce', '')
    word = word.replace('Coordinator', '')
    word = word.replace('Live ', '')
    word = word.replace('Host ', '')
    word = word.replace(' Host', '')
    word = word.replace('Head ', '')
    word = word.replace(' Head', '')
    word = word.replace('Mindfulness', '')
    word = word.replace('Project', '')
    word = word.replace('Residen', '')
    word = word.replace('License', '')
    word = word.replace('Register', '')
    word = word.replace('Information', '')
    word = word.replace('Speaking', '')
    word = word.replace('Public', '')
    word = word.replace('Content', '')
    word = word.replace('Closet', '')
    word = word.replace('Marketing', '')
    word = word.replace('Universal', '')
    word = word.replace('Relation', '')
    word = word.replace('Healthy', '')
    word = word.replace('Compan', '')
    word = word.replace('Automate', '')
    word = word.replace('Grow ', '')
    word = word.replace('Sell', '')
    word = word.replace('Sale', '')
    word = word.replace('Healthcare', '')
    word = word.replace('Resource', '')
    word = word.replace('Master', '')
    word = word.replace('Invest', '')
    word = word.replace('Companies', '')
    word = word.replace('Company', '')
    word = word.replace('Coach', '')
    word = word.replace('Expert', '')
    word = word.replace('Guest', '')
    word = word.replace('Global', '')
    word = word.replace('Retail', '')
    word = word.replace('Guest', '')
    word = word.replace('Success', '')
    word = word.replace('Failure', '')
    word = word.replace('Customer', '')
    word = word.replace('Award', '')
    word = word.replace('Winning', '')
    
    
    word = word.replace("'s", '')
    word = word.replace("’s", '')
    word = word.strip()

    tmp = word.split(' ')
    db = True
    for k in range(len(tmp)//2):
        if tmp[k] != tmp[k + len(tmp)//2]: 
            db = False
            break
    if db: 
        word = ' '.join(tmp[:len(tmp)//2])
        
    word = word.strip().rstrip()
    
    if word != '':
        if word[-1] == "'" or word[-1] == "’": 
            word = word[:-1]
        
    check = True
        
    for each in tmp:
        kt = True
        for letter in each:
            if letter == ' ' or letter == '.' or letter == "'" or letter == '@':
                kt = False
                break
        if kt:
            if letter.isupper():
                check = False
                break
        
    if (content['HOST'][i - deleted].upper() in word.upper()) or \
    (word.upper() in content['HOST'][i - deleted].upper()) or \
    (word.upper() in content['PODCASTS'][i - deleted].upper()) or \
    (content['PODCASTS'][i - deleted].upper() in word.upper()):
        check = False                  
    elif len(word.split(' ')) > 3 or len(word.split(' ')) < 2:
        check = False
    else:
        for chunk in ['THE ',' THE ', ' YOUR ', ' HIS ', ' HER ', ' MY ', ' THEIR ']:
            if chunk in word.upper():
                check = False
                    
        for w in word.split(' '):
            if w != '' and w[0].islower(): 
                check = False 
                break

        tmp = [chr(x) for x in range(ord('A'), ord('Z') + 1)]
            
        for letter in word.upper():
            if letter != ' ' and letter != '.' and letter != "'" and letter != '@':
                if letter not in tmp:
                    check = False
                    break
        
        if check:
            for each in test:
#                 print(word, each)
                if word.upper() in each.upper() or each.upper() in word.upper():
                    check = False
                    break
        
    if check == False: 
        for key in content.keys():
            del content[key][i-deleted]
        deleted += 1
    else:
#         print(word)
        content['PERSON GETTING INTERVIEWED'][i-deleted] = word

In [ ]:
for key in content.keys():
    print(len(content[key]))
print(f'Deleted: {deleted}')